# MCU Battle Analysis and Prediction
## 1. 프로젝트 목표
## 2. 분석할 데이터 수집
#### 2-1. Kaggle에 존재하는 Dataset 수집
####    2-2. **배틀 데이터 수집**
## 3. 데이터 탐구/분석
####    3-1. **캐릭터들의 특징**
####    3-2. **캐릭터들의 스탯**
####    3-3. **배틀 정보**
## 4. Missing Data 처리
####    4-1. characters_info
####    4-2. characters_stat
####    4-3. battles
## 5. 데이터 시각화
####    5-1. **전체적인 표 만들기**
####    5-2. **데이터 시각화**
####    5-3. **그래프 분석 결과**
## 6. 적합한 모델 찾기
### 6-1. K-NN
### 6-2. Decision Tree
### 6-3. Random Forest
### 6-4. Naive Bayes
### 6-5. SVM
## 7. 어벤져스 4 예측
### 7-1. 살아남은 히어로들만 있을 때 전투 결과 예측
### 7-2. 소멸된 히어로들이 돌아왔을 때 전투 결과 예측
### 7-3. 정확도 분석
### 7-4. 결과 분석

# 1. 프로젝트 목표
### 2018년에 개봉한 어벤져스 3를 보고, 그 충격적인 결말로 인해 다음 시리즈가 더 기대되던 중 과연 남은 히어로들이 타노스를 무찌를 수 있을까 궁금했다. (어차피 히어로가 이기겠지만) 그래서 이때까지 축적된 마블 시네마틱 유니버스의 데이터를 사용해서 어벤져스 4에 있을 히어로와 타노스의 전투 결과를 예측하고자 한다. 

# 2. 분석할 데이터 수집
### 우리의 목적은 어벤져스 4에 있을 전투를 예측하는 것이기 때문에 이전의 전투내용과 각 캐릭터별 특징을 수집한다.

## 2-1. Kaggle에 존재하는 Dataset 수집

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("white")

# 우리가 kaggle에서 사용할 데이터 파일들의 목록
import os
print(os.listdir("../input"))

In [ ]:
# 캐릭터들의 특징이 정리되어있는 데이터
characters_info = pd.read_csv("../input/superhero-set/heroes_information.csv")


# 캐릭터들의 스탯이 정리되어있는 데이터
characters_stat = pd.read_csv("../input/characters-stats/charcters_stats.csv")

## 2-2. 배틀 데이터 수집
### 마블 시네마틱 유니버스 팬 위키(http://marvelcinematicuniverse.wikia.com/wiki/Marvel_Cinematic_Universe_Wiki)에서 크롤링 작업 후 정제된 데이터

In [ ]:
# 배틀 정보가 담겨져 있는 파일
battles = pd.read_csv("../input/mcu-battle/mcu_battle.csv")

# 3. 데이터 탐구/분석
### 각 파일을 살펴보고 데이터를 우리가 사용하기 쉽게 정제한다.

## 3-1. 캐릭터들의 특징
### heroes_information.csv 파일을 살펴보자.

In [ ]:
characters_info.head(10)

### Eye color, Hair color, height, publisher, skin color, weight는 전투의 승패와 관련이 없을 것 같으니 지워준다.

In [ ]:
characters_info.drop('Eye color', axis=1, inplace=True)
characters_info.drop('Hair color', axis=1, inplace=True)
characters_info.drop('Height', axis=1, inplace=True)
characters_info.drop('Publisher', axis=1, inplace=True)
characters_info.drop('Skin color', axis=1, inplace=True)
characters_info.drop('Weight', axis=1, inplace=True)
characters_info.head(10)

### Gender, Race, Alignment 를 컴퓨터가 이해하기 쉽게 숫자로 바꿔준다. 
#### Gender = {Female: 1, Male: 0}
#### Race = {Human: 1, Non-Human: 0}
#### Alignment = {good: 1, bad: 0}

In [ ]:
gender_mapping = {'Female': 1, 'Male': 0}
characters_info['Gender'] = characters_info['Gender'].map(gender_mapping)

In [ ]:
characters_info.loc[characters_info.Race != 'Human', 'Race'] = 0
characters_info.loc[characters_info.Race == 'Human', 'Race'] = 1

In [ ]:
alignment_mapping = {'good': 1, 'bad': 0}
characters_info['Alignment'] = characters_info['Alignment'].map(alignment_mapping)

### 첫번째 파일 정제 결과

In [ ]:
characters_info

### 데이터에서 이름 대신 아이디로 다루는 것이 편할 것 같아 이 파일에서 정보를 추출해 딕셔너리를 만든다. 

In [ ]:
name_id_dict = {}                     # {name: id}로 이루어진 딕셔너리
id_num = 0
for name in characters_info['name']:
    name_id_dict[name] = str(id_num)
    id_num += 1
print(name_id_dict)

## 3-2. 캐릭터 스탯 파일
### characters_stat 파일을 살펴보자. 

In [ ]:
characters_stat.head(10)

### 이름을 아이디로 바꿔주고, alignment도 0,1로 바꿔주자. (id가 존재하지 않는 사람이 있는데, 중요한 인물이 아니라 우리가 필요로 하지 않는 사람이므로 무시한다. )

In [ ]:
characters_stat['Name'] = characters_stat['Name'].map(name_id_dict) # 이름 -> id
characters_stat['Alignment'] = characters_stat['Alignment'].map(alignment_mapping) # good = 1, bad = 0
characters_stat.head(10)

## 3-3. 배틀 정보 파일
### battles를 살펴보자. 

In [ ]:
battles.head(10)

### 우리가 분석하고 싶지 않은 필요없는 정보를 지워준다. 

In [ ]:
battles.drop('Name', axis=1, inplace=True)
battles.drop('Date', axis=1, inplace=True)
battles.drop('Location', axis=1, inplace=True)
battles.drop('Commander_SH', axis=1, inplace=True)
battles.drop('Commander_V1', axis=1, inplace=True)
battles.drop('Commander_V2', axis=1, inplace=True)
battles.drop('Unnamed: 13', axis=1, inplace=True)
battles.drop('Unnamed: 14', axis=1, inplace=True)
battles.drop('Unnamed: 15', axis=1, inplace=True)
battles.drop('Unnamed: 16', axis=1, inplace=True)
battles.drop('Unnamed: 17', axis=1, inplace=True)
battles.drop('Unnamed: 18', axis=1, inplace=True)
battles.drop('Villian_2', axis=1, inplace=True)
battles.drop('Casualty_SH', axis=1, inplace=True)
battles.drop('Casualty_V1', axis=1, inplace=True)
battles.drop('Casualty_V2', axis=1, inplace=True)
battles.drop(71, axis=0, inplace=True)
battles

### Result를 히어로가 이겼으면 1, 졌으면 0으로 나타낸다. 

In [ ]:
for i in range(73):
    if battles.iat[i,0] == battles.iat[i,1]:
        battles.iat[i,0] = 1
    else:
        battles.iat[i,0] = 0

### 인물의 이름을 id로 바꾼다. 

In [ ]:
hero_size = []
for i in range(73):
    names = battles.iat[i,1]
    name_list = names.split(',')
    hero_size.append(len(name_list))
    temp = []
    for name in name_list:
        if name in name_id_dict:
            temp.append(str(name_id_dict[name]))
        else:
            pass
    ids = ','.join(temp)
    if ids == '':
        battles.iat[i,1] = None
    else:
        battles.iat[i,1] = ids

In [ ]:
villian_size = []
for i in range(73):
    names = battles.iat[i,2]
    name_list = names.split(',')
    villian_size.append(len(name_list))
    temp = []
    for name in name_list:
        if name in name_id_dict:
            temp.append(str(name_id_dict[name]))
        else:
            pass
    ids = ','.join(temp)
    if ids == '':
        battles.iat[i,2] = None
    else:
        battles.iat[i,2] = ids

# 4. Missing Data
### id가 존재하지 않는 자료가 없는 캐릭터들이 있다.  NaN칸이나 내용이 없는 칸에 유의미한 자료를 넣어준다. 

## 4-1. characters_info
### 먼저 데이터가 없는 칸이 몇개인지 알아본다. 

In [ ]:
characters_info.isnull().sum()

### Gender 에서 중성인 사람 29명, Alignment에서 중립인 사람 31명이 있는 것을 볼 수 있다. 
### 중성인 사람 = 0.5
### 중립인 사람 = 0.5로 넣어준다. 

In [ ]:
characters_info["Gender"].fillna(0.5, inplace=True)

In [ ]:
characters_info["Alignment"].fillna(0.5, inplace=True)

## 4-2. characters_stat
### 먼저 데이터가 없는 칸이 몇개인지 알아본다. 

In [ ]:
characters_stat.isnull().sum()

### name이 없는 사람은 우리에게 필요하지 않은 사람이다. 그러므로 따로 처리하지 않는다. 
### Alignment 가 없는 사람은 중립으로 간주하고 0.5로 넣는다. 

In [ ]:
characters_stat["Alignment"].fillna(0.5, inplace=True)

## 4-3. battles
### 먼저 데이터가 없는 칸이 몇개인지 알아본다. 

In [ ]:
battles.isnull().sum()

### Super Hero 항목에 5개, Villian_1 항목에 30개가 비는 것을 알 수 있다. 
### 모든 인물의 평균값을 특성으로 가지는 가상의 인물을 만들어 집어넣는다. 

In [ ]:
genders = 0
races = 0
alignments = 0
intelli = 0
strength = 0
speed = 0
dur = 0
power = 0
combat = 0
total = 0
for i in range(734):
    genders += characters_info.iat[i, 2]
    races += characters_info.iat[i, 3]
    alignments += characters_info.iat[i,4]
for j in range(611):
    intelli += characters_stat.iat[j,2]
    strength += characters_stat.iat[j,3]
    speed += characters_stat.iat[j,4]
    dur += characters_stat.iat[j,5]
    power += characters_stat.iat[j,6]
    combat += characters_stat.iat[j,7]
    total += characters_stat.iat[j,8]

avg_data1 = {'Unnamed: 0': ['734'],
           'name': ['Average'],
           'Gender': [round(genders/733, 2)],
           'Race': [round(races/733, 2)],
           'Alignment': [round(alignments/733, 2)]}
avg_data2 = {'Name': ['734'],
           'Alignment': [round(alignments/733, 2)],
           'Intelligence': [int(intelli/610)],
           'Strength': [int(strength/610)],
           'Speed': [int(speed/610)],
           'Durability': [int(dur/610)],
           'Power': [int(power/610)],
           'Combat': [int(combat/610)],
           'Total': [int(total/610)]}
print(avg_data1, avg_data2)

In [ ]:
avg_df1 = pd.DataFrame(avg_data1)
avg_df2 = pd.DataFrame(avg_data2)

characters_info = characters_info.append(avg_df1, sort=False)

In [ ]:
characters_stat = characters_stat.append(avg_df2, sort=False)

### 다음 자료를 보면 info, stat 자료에 평균사람의 수치가 포함된 것을 알 수 있다. 

In [ ]:
characters_stat.tail()

In [ ]:
characters_info.tail()

### 빈칸을 Average(734)로 채운다. 

In [ ]:
battles["Super Hero"].fillna('734', inplace=True)
battles["Villian_1"].fillna('734', inplace=True)

In [ ]:
battles

# 5. 배틀 분석
### 모든 수치를 (히어로-빌런)으로 나타낸다 . (히어로-빌런)이 0보다 크면 히어로가 우세한 것이고, 0보다 작으면 빌런이 우세한 것이다. 하나의 수치는 팀의 평균값으로 나타낸다. 

## 5-1. 전체적인 표 만들기

In [ ]:
outcome_list = battles['Result'].tolist()

In [ ]:
size_list = []
for i in hero_size:
    j = villian_size[hero_size.index(i)]
    size_list.append(i-j)

In [ ]:
gender_list = []
race_list = []
alignment_list = []
intel_list = []
strength_list = []
speed_list = []
dur_list = []
power_list = []
combat_list = []
total_list = []

for i in range(73):
    hero_ids = battles.iat[i,1]
    hero_id_list = hero_ids.split(',')
    hero_gender = 0
    hero_race = 0
    hero_alignment = 0
    hero_intel = 0
    hero_strength = 0
    hero_speed = 0
    hero_dur = 0
    hero_power = 0
    hero_combat = 0
    hero_total = 0
    for idd in hero_id_list:
        hero_gender += characters_info.iat[int(idd),2]
        hero_race += characters_info.iat[int(idd),3]
        hero_alignment += characters_info.iat[int(idd),4]
        for j in range(612):
            if characters_stat.iat[j,0] == idd:
                 index = j
        hero_intel += characters_stat.iat[index, 2]
        hero_strength += characters_stat.iat[index, 3]
        hero_speed +=characters_stat.iat[index, 4]
        hero_dur += characters_stat.iat[index, 5]
        hero_power += characters_stat.iat[index, 6]
        hero_combat += characters_stat.iat[index, 7]
        hero_total += characters_stat.iat[index, 8]
    
    vill_ids = battles.iat[i,2]
    vill_id_list = vill_ids.split(',')
    vill_gender = 0
    vill_race = 0
    vill_alignment = 0
    vill_intel = 0
    vill_strength = 0
    vill_speed = 0
    vill_dur = 0
    vill_power = 0
    vill_combat = 0
    vill_total = 0
    for idd in vill_id_list:
        vill_gender += characters_info.iat[int(idd),2]
        vill_race += characters_info.iat[int(idd),3]
        vill_alignment += characters_info.iat[int(idd),4]
        for j in range(612):
            if characters_stat.iat[j,0] == idd:
                 index = j
        vill_intel += characters_stat.iat[index, 2]
        vill_strength += characters_stat.iat[index, 3]
        vill_speed +=characters_stat.iat[index, 4]
        vill_dur += characters_stat.iat[index, 5]
        vill_power += characters_stat.iat[index, 6]
        vill_combat += characters_stat.iat[index, 7]
        vill_total += characters_stat.iat[index, 8]
    
    
    gender_list.append(round(hero_gender/len(hero_id_list) - vill_gender/len(vill_id_list),2))
    race_list.append(round(hero_race/len(hero_id_list) - vill_race/len(vill_id_list),2))
    alignment_list.append(round(hero_alignment/len(hero_id_list) - vill_alignment/len(vill_id_list),2))
    intel_list.append(round(hero_intel/len(hero_id_list) - vill_intel/len(vill_id_list),2))
    strength_list.append(round(hero_strength/len(hero_id_list) - vill_strength/len(vill_id_list),2))
    speed_list.append(round(hero_speed/len(hero_id_list) - vill_speed/len(vill_id_list),2))
    dur_list.append(round(hero_dur/len(hero_id_list) - vill_dur/len(vill_id_list),2))
    power_list.append(round(hero_power/len(hero_id_list) - vill_power/len(vill_id_list),2))
    combat_list.append(round(hero_combat/len(hero_id_list) - vill_combat/len(vill_id_list),2))
    total_list.append(round(hero_total/len(hero_id_list) - vill_total/len(vill_id_list),2))

In [ ]:
new_data = {'Outcome': outcome_list,
            'Size': size_list,
            'Gender': gender_list,
            'Alignment': alignment_list,
            'Race': race_list,
            'Intelligence': intel_list,
            'Strength': strength_list,
            'Speed': speed_list,
            'Durability': dur_list,
            'Power': power_list,
            'Combat': combat_list,
            'Total': total_list
           }
battle_df = pd.DataFrame(new_data)

### 완성된 정리 파일

In [ ]:
battle_df

### 숫자가 너무 복잡해서 눈에 보기 쉽게 나타내기 힘드므로 양수는 1, 음수는 0으로 나타낸다. 
### (히어로가 우세한 수치는 1, 빌런이 우세한 수치는 0)

In [ ]:
new_battle_df = battle_df.copy()

In [ ]:
for i in range(73):
    for j in range(1, 12):
        if new_battle_df.iat[i,j] > 0:
            new_battle_df.iat[i,j] = 1
        elif new_battle_df.iat[i,j] < 0:
            new_battle_df.iat[i,j] = 0
        else:
            new_battle_df.iat[i,j] = 0

In [ ]:
new_battle_df

## 5-2. 데이터 시각화
각 항목들(Size, Gender, Alignment, Race, Intelligence, Strength, Speed, Durability, Power, Combat, Total)이 전투 결과(Outcome)에 미치는 영향이 어느 정도인지 파악한다.

이 때, x축은 전투 결과를(히어로 기준으로 이기면 Win, 지면 Lose) 나타내고 y축은 항목별로 해당 값을 가진 전투 수를 나타낸다.

그래프 형태는 차이를 한 눈에 알아볼 수 있도록 **막대 그래프** 형식을 적용한다.

In [ ]:
def bar_chart(feature):
    Win = new_battle_df[new_battle_df['Outcome']==1][feature].value_counts()
    Lose = new_battle_df[new_battle_df['Outcome']==0][feature].value_counts()
    df = pd.DataFrame([Win, Lose])
    df.index = ['Win','Lose']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

## 5-2 (1) . Size

In [ ]:
bar_chart('Size')

## 5-2 (2). Gender

In [ ]:
bar_chart('Gender')

## 5-2 (3) . Alignment

In [ ]:
bar_chart('Alignment')

## 5-2 (4) . Race

In [ ]:
bar_chart('Race')

## 5-2 (5) . Intelligence

In [ ]:
bar_chart('Intelligence')

## 5-2 (6) . Strength

In [ ]:
bar_chart('Strength')

## 5-2 (7) . Speed

In [ ]:
bar_chart('Speed')

## 5-2 (8) . Durability

In [ ]:
bar_chart('Durability')

 ## 5-2 (9) . Power

In [ ]:
bar_chart('Power')

## 5-2 (10) . Combat

In [ ]:
bar_chart('Combat')

## 5-2 (11) . Total

In [ ]:
bar_chart('Total')

## 5-3. 그래프 분석 결과
위의 총 11개의 그래프를 통해  각 항목들이 전투 결과에 **영향을 미치는 정도**를 파악할 수 있다.

# 6. 적합한 모델 찾기

In [ ]:
target = battle_df['Outcome']
battle_data = battle_df.drop('Outcome', axis=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

## 6-1. K-NN

In [ ]:
clf = KNeighborsClassifier(n_neighbors = 13)
scoring = 'accuracy'
score = cross_val_score(clf, battle_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100, 2)

## 6-2. Decision Tree

In [ ]:
clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, battle_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100, 2)

## 6-3. Random Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=13)
scoring = 'accuracy'
score = cross_val_score(clf, battle_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100, 2)

## 6-4. Naive Bayes

In [ ]:
clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(clf, battle_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100, 2)

## 6-5. SVM

In [ ]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, battle_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100,2)

### Naive Bayes가 점수가 가장 높게 나왔으므로 이 모델로 예측을 진행한다. 

# 7. 어벤져스 4 전투내용 예측 (어벤져스 3 스포일러)
### 어벤져스 3 결말에 살아남은 히어로의 명단은 ['Iron Man', 'Nebula', 'Captain America', 'Black Widow', 'Thor', 'Hulk', 'War Machine', 'Rocket the Racoon', 'Okoye', 'Ant-Man'] 이다. 
### 여기서 두 가지로 생각해 볼 수 있다. 어떠한 방법으로든 소멸된 어벤져스 멤버들이 살아 돌아오는 경우와, 남은 멤버만 타노스와 싸우는 경우를 생각해볼 수 있다. 

In [ ]:
survived = ['Iron Man', 'Nebula', 'Captain America', 'Black Widow', 'Thor', 'Hulk', 'War Machine', 'Rocket Raccoon', 'Ant-Man']
hero_all = ['Iron Man', 'Nebula', 'Captain America', 'Black Widow', 'Thor', 'Hulk', 'War Machine', 'Rocket Raccoon', 'Ant-Man', 'Spider-Man', 'Star-Lord', 'Drax the Destroyer', 'Mantis', 'Doctor Strange', 'Falcon', 'Scarlet Witch', 'Vision', 'Groot', 'Black Panther', 'Winter Soldier', 'Nick Fury']
villian = ['Thanos']

survived_id = []
hero_all_id = []
villian_id = []
for char in survived:
    survived_id.append(name_id_dict[char])
for char in hero_all:
    hero_all_id.append(name_id_dict[char])
for char in villian:
    villian_id.append(name_id_dict[char])

### 살아남은 히어로들만 싸웠을 때의 정보와 모든 히어로들이 싸웠을 때의 정보를 수집해서 리스트로 나타낸다. 

In [ ]:
survived_list = [len(survived)-1]
hero_all_list = [len(hero_all)-1]

hero_gender = 0
hero_race = 0
hero_alignment = 0
hero_intel = 0
hero_strength = 0
hero_speed = 0
hero_dur = 0
hero_power = 0
hero_combat = 0
hero_total = 0
for idd in survived_list:
    hero_gender += characters_info.iat[int(idd),2]
    hero_race += characters_info.iat[int(idd),3]
    hero_alignment += characters_info.iat[int(idd),4]
    for j in range(612):
        if characters_stat.iat[j,0] == idd:
             index = j
    hero_intel += characters_stat.iat[index, 2]
    hero_strength += characters_stat.iat[index, 3]
    hero_speed += characters_stat.iat[index, 4]
    hero_dur += characters_stat.iat[index, 5]
    hero_power += characters_stat.iat[index, 6]
    hero_combat += characters_stat.iat[index, 7]
    hero_total += characters_stat.iat[index, 8]
    
idd = villian_id[0]
vill_gender = characters_info.iat[int(idd),2]
vill_race = characters_info.iat[int(idd),3]
vill_alignment = characters_info.iat[int(idd),4]
for j in range(612):
        if characters_stat.iat[j,0] == idd:
             index = j
vill_intel = characters_stat.iat[index, 2]
vill_strength = characters_stat.iat[index, 3]
vill_speed = characters_stat.iat[index, 4]
vill_dur = characters_stat.iat[index, 5]
vill_power = characters_stat.iat[index, 6]
vill_combat = characters_stat.iat[index, 7]
vill_total = characters_stat.iat[index, 8]
    
survived_list.append(round(hero_gender/len(survived_list) - vill_gender,2))
survived_list.append(round(hero_race/len(survived_list) - vill_race,2))
survived_list.append(round(hero_alignment/len(survived_list) - vill_alignment,2))
survived_list.append(round(hero_intel/len(survived_list) - vill_intel,2))
survived_list.append(round(hero_strength/len(survived_list) - vill_strength,2))
survived_list.append(round(hero_speed/len(survived_list) - vill_speed,2))
survived_list.append(round(hero_dur/len(survived_list) - vill_dur,2))
survived_list.append(round(hero_power/len(survived_list) - vill_power,2))
survived_list.append(round(hero_combat/len(survived_list) - vill_combat,2))
survived_list.append(round(hero_total/len(survived_list) - vill_total,2))
    
hero_gender = 0
hero_race = 0
hero_alignment = 0
hero_intel = 0
hero_strength = 0
hero_speed = 0
hero_dur = 0
hero_power = 0
hero_combat = 0
hero_total = 0
for idd in hero_all_list:
    hero_gender += characters_info.iat[int(idd),2]
    hero_race += characters_info.iat[int(idd),3]
    hero_alignment += characters_info.iat[int(idd),4]
    for j in range(612):
        if characters_stat.iat[j,0] == idd:
             index = j
    hero_intel += characters_stat.iat[index, 2]
    hero_strength += characters_stat.iat[index, 3]
    hero_speed += characters_stat.iat[index, 4]
    hero_dur += characters_stat.iat[index, 5]
    hero_power += characters_stat.iat[index, 6]
    hero_combat += characters_stat.iat[index, 7]
    hero_total += characters_stat.iat[index, 8]

idd = villian_id[0]
vill_gender = characters_info.iat[int(idd),2]
vill_race = characters_info.iat[int(idd),3]
vill_alignment = characters_info.iat[int(idd),4]
for j in range(612):
    if characters_stat.iat[j,0] == idd:
        index = j
vill_intel = characters_stat.iat[index, 2]
vill_strength = characters_stat.iat[index, 3]
vill_speed = characters_stat.iat[index, 4]
vill_dur = characters_stat.iat[index, 5]
vill_power = characters_stat.iat[index, 6]
vill_combat = characters_stat.iat[index, 7]
vill_total = characters_stat.iat[index, 8]


hero_all_list.append(round(hero_gender/len(survived_list) - vill_gender,2))
hero_all_list.append(round(hero_race/len(survived_list) - vill_race,2))
hero_all_list.append(round(hero_alignment/len(survived_list) - vill_alignment,2))
hero_all_list.append(round(hero_intel/len(survived_list) - vill_intel,2))
hero_all_list.append(round(hero_strength/len(survived_list) - vill_strength,2))
hero_all_list.append(round(hero_speed/len(survived_list) - vill_speed,2))
hero_all_list.append(round(hero_dur/len(survived_list) - vill_dur,2))
hero_all_list.append(round(hero_power/len(survived_list) - vill_power,2))
hero_all_list.append(round(hero_combat/len(survived_list) - vill_combat,2))
hero_all_list.append(round(hero_total/len(survived_list) - vill_total,2))
    

In [ ]:
X = battle_data
Y = target

## 7-1. 살아남은 히어로들만 있을 때 전투 결과 예측

In [ ]:
clf = GaussianNB()
clf.fit(X, Y)
target_pred1 = clf.predict([survived_list])

In [ ]:
print(target_pred1)

### 결과가 0으로 나온 것을 보아 질 확률이 높음을 알 수 있다. 

## 7-2. 소멸된 히어로들이 살아돌아왔을 때 전투 결과 예측

In [ ]:
target_pred2 = clf.predict([hero_all_list])

In [ ]:
print(target_pred2)

### 결과가 0으로 나온 것을 보아 질 확률이 높음을 알 수 있다. 

## 7-3. 정확도

In [ ]:
scoring = 'accuracy'
score = cross_val_score(clf, X, Y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100, 2)

### 이 모델의 정확도는 약 70%이다. 

## 7-4. 결과분석
### 아무래도 타노스가 너무 세다 보니 히어로가 이길 확률은 높지 않은 것으로 나타났다. 이 모델의 정확도가 70퍼센트인 걸 감안하면, 나머지 30퍼센트의 가능성으로 타노스를 이길 수 있지 않을까?
### 정확도를 그래도 80퍼센트까지는 기대했는데, 데이터가 별로인 모양인지 기대보다는 낮게 나타났다. 정확도를 올리기 위해서는 어떻게 데이터를 손봐야 할지 궁금하다. 단순히 데이터를 분석/예측하는 프로젝트도 좋지만, 다음엔 데이터의 정확도를 올리기 위한 방법을 연구하는 프로젝트도 하면 좋을 것 같다. 